

<img src="http://www.exalumnos.usm.cl/wp-content/uploads/2015/06/ISOTIPO-Color.jpg" title="Title text" width="20%" />

<hr style="height:2px;border:none"/>
<h1 align='center'> INF-479 Reconocimiento de patrones en minería de datos </h1>

<H2 align='center'> Tarea 3 - Sistemas Recomendadores </H2>
<H3 align='center'> Lucio Fondón Rebolledo - 201773610-0</H3>
<H3 align='center'> Francisco Reyes Jainaga - 201773529-5</H3>
<hr style="height:2px;border:none"/>

# Instrucciones

1. La tarea debe ser desarrollada en parejas.  
2. Los medios de consultas serán Aula y el canal de Discord oficial del curso. 
3. La tarea debe ser realizada en Jupyter Notebook (Python3) utilizando este archivo como base. 
4. Se evaluará la correcta implementación de los algoritmos de filtrado colaborativo aplicados a los datasets escogido por cada grupo, como también la comprensión de los fundamentos teóricos, ventajas y desventajas de cada técnica. 
5. El archivo de entrega debe denominarse T3_Apellido1_Apellido2.ipynb. De no respetarse este formato existirá un descuento de 30 puntos. 
6. La fecha de entrega es el lunes 2 de agosto a las 23:55 hrs, posterior a esto se descontará 1 punto por cada minuto de atraso.  

In [ ]:
#Librerías 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import NearestNeighbors


## Datasets Tarea

Para la tarea tendrán a su disposición los siguientes 3 datasets, <b> de las cuales tendrán que escoger 2 para realizar la implementación de su tarea. </b>

   ### 1. MovieLens: 
   Es uno de los datasets más populares de recomendación y recopila las preferencias de usuarios de internet con respecto a películas que son evaluadas de 0 a 5 estrellas. Este dataset ha sido utilizado en diversos estudios de investigación en áreas como la recomendación personalizada y la psicología social.  
* <b>Archivos: </b>
    - ML_ratings.csv: contiene los ratings dados por usuarios a películas. Se compone por las columnas user_id, movie_id y rating. 
    - movies.csv: contiene la metadata sobre las películas. Incluye las columnas movie_id, title y genres (donde se encuentran los géneros de la película separados por "|"). 
 
* <b>Cantidad de ratings</b>: 100836
* <b>Cantidad de usuarios</b>: 610
* <b>Cantidad de películas</b>: 9724
* <b>Calificación</b> 0 a 5 (valores enteros) 

Referencias: [Movielens Dataset](https://grouplens.org/datasets/movielens/) 
   

In [ ]:
df_ML_ratings = pd.read_csv("ML_ratings.csv")
df_movies = pd.read_csv("movies.csv")

display(df_ML_ratings)
display(df_movies)

### 2. Book-Crossing: 
Es un datasets de ratings de libros recolectado por Cai-Nicolas Ziegler desde la <i>Book-Crossing community</i>. El dataset original contiene 1,149,780 ratings generados por 278,858 usuarios a 271,379 libros. 

* <b>Archivos</b>: 
    - BX_ratings.csv: contiene los ratings dados por usuarios a diferentes libros. Se compone de las columnas user_id, ISBN (identificador para los libros) y rating.
    - books.csv: contiene los títulos de libros asociados a los códigos ISBN (utilizados como item_id). Se compone de las columnas ISBN y title. 

* <b>Cantidad de ratings</b>: 359263
* <b>Cantidad de usuarios</b>: 10775
* <b>Cantidad de libros</b>: 10773
* <b>Calificación</b>: 0 a 10 (valores enteros)

Referencias: [Book-Crossing Dataset](http://www2.informatik.uni-freiburg.de/~cziegler/BX/)

In [ ]:
df_BX_ratings = pd.read_csv("BX_ratings.csv")
df_books = pd.read_csv("books.csv")

display(df_BX_ratings)
display(df_books)

### 3. Jester:     
Es un dataset desarrollado por Ken Goldberg y su equipo en la Universidad de Berkeley, el cual contiene alrededor de 6 millones de ratings con respecto a 150 chistes cortos. 
* <b>Archivos</b>:
    - JT_ratings.csv: contiene los ratings dados por usuarios a diferentes chistes cortos. Se compone por las columnas joke_id, user_id y rating. 
    - jokes.csv: contiene 

* <b>Cantidad de ratings</b>: 199900
* <b>Cantidad de usuarios</b>: 1999
* <b>Cantidad de chistes</b>: 100 
* <b>Calificación</b>: -10 a 10 (valores reales)
    
Referencias: [Jester Dataset](http://eigentaste.berkeley.edu/dataset/)

In [ ]:
df_JT_ratings = pd.read_csv("JT_ratings.csv")
df_jokes = pd.read_csv("jokes.csv")

display(df_JT_ratings)
display(df_jokes)

## Instrucciones

<img src="https://media.springernature.com/lw685/springer-static/image/art%3A10.1007%2Fs11227-020-03266-2/MediaObjects/11227_2020_3266_Fig1_HTML.png" title="Title text" width="60%" />
<center> <i> Figura 1. Collaborative Filtering. </i> </center>

## 1. User based Collaborative Filtering (40 puntos) 

La primera parte de la tarea constará de las siguientes secciones: 
1. Implementación de un sistema de recomendación de filtro colaborativo basado en usuarios utilizando los k vecinos más cercanos. Para esta parte, se debe utilizar <b> similaridad coseno </b> como medida de similaridad entre el usuario objetivo y el resto de usuarios. Además, el parámetro k debe ser escogido por ustedes. 

2. Se ingresa nuevo usuario al sistema, se le pide que califique 10 productos (a elección) y a partir de eso se le realiza la recomendación de 5 productos que no ha calificado. Es importante que en su procedimiento se muestren las ids de los k vecinos más cercanos.  

3. Concluir y responder las siguientes preguntas: 

   >a. ¿Cuáles fueron las 5 recomendaciones obtenidas? Analice y concluya respecto a estos resultados.  
    
   >b. ¿Qué cantidad de vecinos cercanos (k) se escogió para la recomendación? ¿En qué influye la elección de este parámetro? ¿Qué sucede a medida que aumenta este parámetro?
    
   >c. ¿Cuál era el porcentaje de <i>sparsity</i> de la matriz usuarios-items? ¿Cuáles son las desventajas de este enfoque? 


<i>Importante: Esto debe ser realizado para ambos datasets escogidos</i>


## Desarrollo P1

Primero, definiremos funciones que nos servirán de ayuda para los dos datasets elegidos (__MovieLens__ y __Jester__). Definiremos una función que nos retorne los $k$ id's de los usuarios más similares respecto a un usuario objetivo y otra función que realizará la recomendación de las $n$ mejores películas para el usuario objetivo

In [ ]:
# Creamos una función que nos retorne los usuarios más similares respecto a un usuario objetivo
# Se utilizará (como es pedido) la similaridad coseno como métrica

"""
input
user_id  : (int) id del usuario objetivo 
matrix   : (2D array) matriz de ratings entre los usuarios y los items con ratings
k        : (int) k vecinos más cercanos

output
users    : (1D array) arreglo con las id's de los k usuarios más cercanos al usuario objetivo
"""

def similar_users(user_id, matrix, k=3):
    user = matrix[matrix.index == user_id] # ratings usuario objetivo
    other_users = matrix[matrix.index != user_id] # ratings resto de usuarios
    
    # Calculamos la similaridad coseno entre el usuario objetivo y el resto de ellos
    similarities = cosine_similarity(user,other_users)[0].tolist()
    
    indices = other_users.index.tolist()
    
    # Diccionario donde la llave es el id de un usuario distinto al objetivo
    # el valor es la similaridad coseno del usuario con el usuario objetivo
    index_similarity = dict(zip(indices, similarities))
        
    # Ordenamos por similaridad de mayor a menor
    index_similarity_sorted = sorted(index_similarity.items(), key=lambda x:x[1], reverse=True)
    
    # Escogemos los k primeros vecinos más cercanos (son los con mayor similitud con el usuario objetivo)
    top_users_similarities = index_similarity_sorted[:k]
    
    # Obtenemos los id de los usuarios
    users = [u[0] for u in top_users_similarities]
    return users

# Creamos función que nos retorne los n items con mejor recomendación para un usuario objetivo

"""
input
user_id            : (int) id del usuario objetivo 
similar_user_ids   : (1D array) arreglo que contiene las id's de usuarios similares
matrix             : (2D array) matriz de ratings entre los usuarios y los items con ratings
dataset            : (string) string que corresponde al nombre del dataset que se utiliza
n                  : (int) n items a recomendar para el usuario objetivo user_id

output
items_prediction   : (array) arreglo con las mejores n predicciones para el usuario objetivo
"""

def recommend_item(user_id, similar_user_ids, matrix, dataset, n=5):
    # Obtenemos los vectores de los usuarios similares al usuario objetivo obtenidos con la función similar_users
    similar_users = matrix[matrix.index.isin(similar_user_ids)]

    # Calculamos el rating de los k usuarios más cercanos
    similar_users = similar_users.mean(axis=0)
    similar_users_df = pd.DataFrame(similar_users, columns=['mean'])
        
    # Obtenemos el vector del usuario objetivo
    user_df = matrix[matrix.index == user_id]
    user_df_transposed = user_df.transpose()
    user_df_transposed.columns = ['rating']
    
    # Obtenemos los items que el usuario objetivo NO ha rateado
    user_df_transposed = user_df_transposed[user_df_transposed['rating']==0]
    items_not_rated = user_df_transposed.index.tolist()
        
    # Filtramos el rating promedio de los usuarios de los items no rateados por el usuario objetivo
    similar_users_df_filtered = similar_users_df[similar_users_df.index.isin(items_not_rated)]
   
    # Ordenamos el dataframe de mayor a menor rating y obtenemos los n primeros
    similar_users_df_ordered = similar_users_df.sort_values(by=['mean'], ascending=False)
    top_n_items = similar_users_df_ordered.head(n)
    top_n_items_indices = top_n_items.index.tolist()
    
    # Dependiendo del dataset usado, se utiliza el nombre correspondiente
    item_id = ''
    if dataset == 'movielens':
        item_id = 'movie_id'
        df_final = df_movies
    elif dataset == 'jester':
        item_id = 'joke_id'
        df_final = df_jokes
    
    # Se consulta al dataset con los top n items seleccionados
    items_prediction = df_final[df_final[item_id].isin(top_n_items_indices)]
    
    return items_prediction

## Desarrollo P2

Ahora, debemos ingresar un nuevo usuario al sistema y que califique 10 productos. Para esto, se realizará una función que realizará automáticamente esta calificación para los dos datasets escogidos. Éstos se escogerán de manera al azar, por lo que se seteará una semilla para poder replicar resultados y que la revisión sea más adecuada y fácil de realizar

In [ ]:
"""
input
dataset : (string) string que corresponde al nombre del dataset que se utiliza

output
df      : (pandas dataframe) dataframe con las 10 calificaciones agregadas
"""

def rate_10_products(dataset):
    np.random.seed(0) # Se setea una semilla para poder replicar resultados
    if dataset == 'movielens':
        df = df_ML_ratings.copy()
        
        # Sacamos el id nuevo que tendrá el usuario, se le sumará +1 al último que se encuentre en el df
        new_user_id = df.iloc[-1]['user_id'] + 1
        
        # Obtenemos todos los id de las películas
        movie_ids = df['movie_id'].unique()
        
        # Seleccionamos al azar 10 películas para calificarlas
        random_movies = np.random.choice(movie_ids, size=10, replace=False)
        
        # Generamos ratings de manera aleatoria en el rango de 0-5
        random_ratings = np.random.randint(0, 6, size=10)
        
        # Diccionario con todos los valores generados
        random_final = {
            'user_id' : [new_user_id] * 10,
            'movie_id' : random_movies,
            'rating' : random_ratings,
        }
        
        # Generamos un dataframe para los datos del usuario nuevo
        df_random_ratings = pd.DataFrame(random_final)
        
        # Printearmos para ver qué películas calificó el usuario
        print("El usuario con user_id de " + str(new_user_id) + " calificó las siguientes películas:")
        
        for index, row in df_random_ratings.iterrows():
            movie =  df_movies[df_movies['movie_id'] == row['movie_id']]['title'].item()
            rating = row['rating']
            print("Película: " + movie + " con rating " + str(rating))
        
        # Agregamos los datos al df original
        df = df.append(df_random_ratings)
        
        # Retornamos el nuevo df con los datos del nuevo usuario
        return df
        
    elif dataset == 'jester':
        df = df_JT_ratings.copy()
        
        # Sacamos el id nuevo que tendrá el usuario, se le sumará +1 al último que se encuentre en el df
        new_user_id = df.iloc[-1]['user_id'] + 1
        
        # Obtenemos todos los id de las bromas
        joke_ids = df['joke_id'].unique()
        
        # Seleccionamos al azar 10 bromas para calificarlas
        random_jokes = np.random.choice(joke_ids, size=10, replace=False)
        
        # Generamos ratings de manera aleatoria en el rango de -10.0 a 10.0
        random_ratings = np.random.uniform(-10.0, 10.0, 10).round(decimals=2)
        
        # Diccionario con todos los valores generados
        random_final = {
            'joke_id' : random_jokes,
            'user_id' : [new_user_id] * 10,
            'rating' : random_ratings,
        }
        
        # Generamos un dataframe para los datos del usuario nuevo
        df_random_ratings = pd.DataFrame(random_final)
        
        # Printearmos para ver qué películas calificó el usuario
        print("El usuario con user_id de " + str(new_user_id) + " calificó las siguientes bromas: ")
        
        for index, row in df_random_ratings.iterrows():
            movie =  df_jokes[df_jokes['joke_id'] == row['joke_id']]['joke_text'].item()
            rating = row['rating']
            print("Broma: " + movie + "\n Con rating " + str(rating) +"\n")
            print("-" * 60)
        # Agregamos los datos al df original
        df = df.append(df_random_ratings)
        
        # Retornamos el nuevo df con los datos del nuevo usuario
        return df
            

### MovieLens

Primero, utilizaremos la función $\texttt{rate_10_products}$ para crear un nuevo usuario al azar y que califique aleatoriamente 10 películas

In [ ]:
movies_with_new_rating = rate_10_products('movielens')
movies_with_new_rating

Luego, debemos generar la matriz de ratings entre los usuarios y las películas rateadas, de tal forma que quede una matriz cuadrada $n x n$, en donde $n$ serían la cantidad de usuarios en el dataset, y los valores de la matriz será el rating del usuario $i$ con la película $j$

In [ ]:
rating_matrix_movies = movies_with_new_rating.pivot_table(index = 'user_id',columns = 'movie_id', values = 'rating').fillna(0)
rating_matrix_movies

Ahora, solo debemos utilizar las funciones realizadas anteriormente para poder obtener las 5 recomendaciones pedidas. Se probarán con distintos valores de $k$, con el fin de poder concluir respecto a la elección de $k$

### k = 3

In [ ]:
current_user = 611
similar_user_indices = similar_users(current_user, rating_matrix_movies, k=3)
print(similar_user_indices)
recommend_item(current_user, similar_user_indices, rating_matrix_movies, 'movielens')


### k = 5

In [ ]:
current_user = 611
similar_user_indices = similar_users(current_user, rating_matrix_movies, k=5)
print(similar_user_indices)
recommend_item(current_user, similar_user_indices, rating_matrix_movies, 'movielens')


### k = 10


In [ ]:
current_user = 611
similar_user_indices = similar_users(current_user, rating_matrix_movies, k=10)
print(similar_user_indices)
recommend_item(current_user, similar_user_indices, rating_matrix_movies, 'movielens')



### k = 30


In [ ]:
current_user = 611
similar_user_indices = similar_users(current_user, rating_matrix_movies, k=30)
print(similar_user_indices)
recommend_item(current_user, similar_user_indices, rating_matrix_movies, 'movielens')


### Jester

Análogo a lo anterior, se realizan los mismos pasos anteriores

In [ ]:
jokes_with_new_rating = rate_10_products('jester')
jokes_with_new_rating

In [ ]:
rating_matrix_jokes = jokes_with_new_rating.pivot_table(index = 'user_id',columns = 'joke_id', values = 'rating').fillna(0)
rating_matrix_jokes

### k = 3

In [ ]:
current_user = 2000
similar_user_indices = similar_users(current_user, rating_matrix_jokes, k=3)
print(similar_user_indices)
recommend_item(current_user, similar_user_indices, rating_matrix_jokes, 'jester')


### k = 5

In [ ]:
current_user = 2000
similar_user_indices = similar_users(current_user, rating_matrix_jokes, k=5)
print(similar_user_indices)
recommend_item(current_user, similar_user_indices, rating_matrix_jokes, 'jester')


### k = 10

In [ ]:
current_user = 2000
similar_user_indices = similar_users(current_user, rating_matrix_jokes, k=10)
print(similar_user_indices)
recommend_item(current_user, similar_user_indices, rating_matrix_jokes, 'jester')


### k = 30


In [ ]:
current_user = 2000
similar_user_indices = similar_users(current_user, rating_matrix_jokes, k=30)
print(similar_user_indices)
recommend_item(current_user, similar_user_indices, rating_matrix_jokes, 'jester')

### k = 50


In [ ]:
current_user = 2000
similar_user_indices = similar_users(current_user, rating_matrix_jokes, k=50)
print(similar_user_indices)
recommend_item(current_user, similar_user_indices, rating_matrix_jokes, 'jester')

## Desarrollo P3

3. Concluir y responder las siguientes preguntas: 

   >a. ¿Cuáles fueron las 5 recomendaciones obtenidas? Analice y concluya respecto a estos resultados.  
    
   >b. ¿Qué cantidad de vecinos cercanos (k) se escogió para la recomendación? ¿En qué influye la elección de este parámetro? ¿Qué sucede a medida que aumenta este parámetro?
    
   >c. ¿Cuál era el porcentaje de <i>sparsity</i> de la matriz usuarios-items? ¿Cuáles son las desventajas de este enfoque? 


### a.- 
Con un k inicial de 3 vecinos los resultados para ambos dataset son: 

In [ ]:
current_user = 611
similar_user_indices = similar_users(current_user, rating_matrix_movies, k=3)
print(similar_user_indices)
recommend_item(current_user, similar_user_indices, rating_matrix_movies, 'movielens')


Al comparar las cinco sugerencias locales, podemos ver una tendencia o inclinación por lo generos, como son Comedy, Drama y Romance, las cuales salen en casi todas las películas recomendadas. Por lo que podemos interpretar esto como los posibles gustos del usuario ingresado según por lo que el mismo califico dentro de sus 10 evaluaciones.

In [ ]:
current_user = 2000
similar_user_indices = similar_users(current_user, rating_matrix_jokes, k=3)
print(similar_user_indices)
recommend_item(current_user, similar_user_indices, rating_matrix_jokes, 'jester')

### b.-  
Luego de una variacíon de los k vecinos a comparar al momento de entregar las recomendaciones, podemos ver que para ambos dataset al momento de aumentar el k existen recomendaciones que siguen apareciendo al ser de las mejores 5 opciones locales en esos momentos, como en el caso de la película Star Wars: Episode IV - A New Hope (1977), la cual aparece repetidas veces. Como tambien sugerencias que solo salen una vez, como en el dataset de jokes con la siguiente broma, 49	A guy goes into confession and says to the pri..., la cual solo sale al momento de comparar 30 vecinos cercanos, probablemente eso ocurra ya que estaba bastante alejada de la persona a comparar, pero puede que si aumentamos la cantidad de vecinos esta mantenga su aparición como sugerencia. (Las 5 sugerencias de cada dataset con variaciones de vecinos estan señaladas en las celdas de arriba).

### c.-

   > __R:__ El <i>sparsity</i> de una matriz $A$ se define como:
   $$sparsity = 1 - \frac{cantidad\_no\_ceros(A)}{total\_elementos(A)}$$ 
   
   > entonces, crearemos una función que nos ayudará a calcular lo anterior



In [ ]:
"""
input
A        : (pandas datafrane) matriz de ratings en forma de dataframe

output
sparsity : (double) valor del sparsity de la matriz
"""
def sparsity(A):
    # Calculamos la cantidad de elementos no nulos de A
    cantidad_no_ceros = np.count_nonzero(A)
    
    # Calculamos el total de elementos de A
    total_elementos = A.size
    
    # Calculamos la sparsity
    sparsity = 1 - cantidad_no_ceros/total_elementos
    
    return sparsity

   
   > __Sparsity para MovieLens:__


In [ ]:
print("El porcentaje de sparsity de la matriz del dataset MovieLens es: " + str(sparsity(rating_matrix_movies)* 100) + "%")

   > __Sparsity para Jester:__


In [ ]:
print("El porcentaje de sparsity de la matriz del dataset Jester es: " + str(sparsity(rating_matrix_jokes)* 100) + "%")

> El <i>sparsity</i> de la matriz de ratings para ambos datasets escogidos da diametralmente distinto, obteniendóse para MovieLens un $sparsity\_movielens \approx 98\%$ y $sparsity\_jester \approx 0.36\%$. Lo anterior indica que, comparando los dos resultados obtenidos, dentro del dataset de __MovieLens__ existe una gran cantidad de usuarios (casi en su totalidad) que han calificado muy pocas películas (y estamos hablando de un dataset de más de 9700 películas), por lo que existirán muchos valores nulos que no aportarán mucho y es más, perjudicarán la precisión de la predicción de las recomendaciones, esto principalmente debido a que al tener películas que tienen por ejemplo un solo usuario que la haya calificado, y que este usuario tenga similaridad con el usuario objetivo, esto podría tender a obtener una recomendación errónea debido a que se está basando en una sola calificación para hacer el cómputo, no así como ocurre con el dataset __Jester__, ya que este presenta un porcentaje casi nulo de sparsity, lo que indica que casi la totalidad de los usuarios contenidos dentro del dataset han calificado la mayoría de las bromas que contiene el dataset (podría de hecho tomarse como una <i>matriz densa</i>, o sea, que la mayoría de sus elementos son no nulos), por lo que, al tener más información con la qué trabajar, la predicción tiene a ser más precisa y confiable.


## 2. Item based Collaborative Filtering (40 puntos) 

La segunda parte de la tarea constará de las siguientes secciones: 
1. Implementación de un sistema de recomendación de filtro colaborativo basado en ítems utilizando los k ítems más cercanos. Para esta parte, se debe utilizar <b> similaridad coseno </b> como medida de similaridad entre los ítems. Además, el parámetro k debe ser escogido por ustedes. 

2. Se ingresa nuevo usuario al sistema, se le pide que califique 10 productos (a elección) y a partir de eso se le realiza la recomendación de 5 productos que no ha calificado. Para poder llevar a cabo la recomendación recordar los siguientes pasos:
    >a. Se debe generar la matriz de similaridad entre productos basada en la similaridad coseno.
    
    >b. Para realizar la recomendación, se debe predecir el rating de todos los productos que el usuario nuevo del sistema no haya calificado aún. Para esto, se deben obtener los k ítems más cercanos al ítem a predecir y se debe predecir su rating en base a la <i> weighted sum </i> de los ratings de dichos k ítems. 
    
    >c. Una vez predecidos los ratings para todos los productos sin calificación, se deben recomendar al usuario los 5 productos con mayor rating predecido. 

3. Concluir y responder las siguientes preguntas: 

   >a. ¿Cuáles fueron las 5 recomendaciones obtenidas? Analice y concluya respecto a estos resultados.  
    
   >b. ¿Qué cantidad de vecinos cercanos (k) se escogió para la recomendación? ¿En qué influye la elección de este parámetro? ¿Qué sucede a medida que aumento este parámetro?
    
   >c. ¿Cuáles son las ventajas de este enfoque respecto al anterior? 


<i>Importante: Esto debe ser realizado para ambos datasets escogidos</i>

## Desarrollo P1

Utilizaremos el módulo __NearestNeighbors__ de sklearn, con la métrica de distana coseno para poder predecir luego los ratings del usuario que se ingresará. Se utilizará en primera instancia los 3 vecinos más cercanos y luego ir variando el valor para concluir

In [ ]:
def item_based_knn(rating_item_matrix, number_neighbors=3):
    knn = NearestNeighbors(metric='cosine', algorithm='brute')
    knn.fit(rating_item_matrix.values)
    
    distances, indices = knn.kneighbors(rating_item_matrix.values, n_neighbors=number_neighbors)
    
    return distances, indices

## Desarrollo P2

In [ ]:
def get_predicted(rating_matrix, user, number_neighbors=3):
    copia_aux = rating_matrix.copy()    
    user_index = rating_matrix.columns.tolist().index(user)
        
    distances, indices = item_based_knn(copia_aux,number_neighbors)
    #empieza la busqueda de sugerencias
    for m,t in list(enumerate(copia_aux.index)):

          # se encuentran libros sin calificar de usuario
            if copia_aux.iloc[m, user_index] == 0:
                similar_items = indices[m].tolist()
                items_distances = distances[m].tolist()

            #se saca el libro de las similitudes. 
            if m in similar_items:
                item_id = similar_items.index(m)
                similar_items.remove(m)
                items_distances.pop(item_id) 

            # si existen muchos ceros a vecinos cercanos se sacan los vecinos mas lejanos.
            else:
                similar_items = similar_items[:number_neighbors-1]
                items_distances = items_distances[:number_neighbors-1]


            item_similarity = [1-x for x in items_distances]
            item_similarity_copy = item_similarity.copy()
            nominator = 0

            # para cada libro similar
            for s in range(0, len(item_similarity)):

              # vemos si valor es cero
                if copia_aux.iloc[similar_items[s], user_index] == 0:

                # de ser cero se ignora
                    if len(item_similarity_copy) == (number_neighbors - 1):
                        item_similarity_copy.pop(s)

                    else:
                        item_similarity_copy.pop(s-(len(item_similarity)-len(item_similarity_copy)))

                else:
                    nominator = nominator + item_similarity[s]*copia_aux.iloc[similar_items[s],user_index]
            if len(item_similarity_copy) > 0:
                if sum(item_similarity_copy) > 0:
                    predicted_r = nominator/sum(item_similarity_copy)
                else:
                    predicted_r = 0
            else:
                predicted_r = 0
            copia_aux.iloc[m,user_index] = predicted_r
            
    #la copia del la matriz, copia_aux, actualiza todas las predicciones para el usuario user       
    return copia_aux

def item_recom(rating_matrix, user, num_recommended_items,dataset, number_neighbors=3):
    copia_aux = get_predicted(rating_matrix, user, number_neighbors)
    
    recommended_items = []

    for m in rating_matrix[rating_matrix[user] == 0].index.tolist():

        index_rating = rating_matrix.index.tolist().index(m)
        predicted_rating = copia_aux.iloc[index_rating, copia_aux.columns.tolist().index(user)]
        recommended_items.append((m, predicted_rating))

    sorted_rm = sorted(recommended_items, key=lambda x:x[1], reverse=True)
      
    if dataset == 'movielens':
        print('Películas recomendadas para el usuario de id: {}\n'.format(user))
        rank = 1
        for recommended_items in sorted_rm[:num_recommended_items]:
            movie =  df_movies[df_movies['movie_id'] == recommended_items[0]]['title'].item()
            rating = recommended_items[1]        
            print('{}: {} - predicción: {}'.format(rank, movie, rating))
            rank = rank + 1
            
    elif dataset == 'jester':
        print('Chistes recomendados para el usuario de id: {}\n'.format(user))
        rank = 1
        for recommended_items in sorted_rm[:num_recommended_items]:
            joke =  df_jokes[df_jokes['joke_id'] == recommended_items[0]]['joke_text'].item()
            rating = recommended_items[1]        
            print('{}: {} - predicción: {}'.format(rank, joke, rating))
            rank = rank + 1
            print("-"*80)

### MovieLens

Igual que en el UBCF, utilizaremos las funciones creadas ahí para que un usuario nuevo califique 5 películas, para luego utilizar el item_based_knn sobre el nuevo df, entonces:

In [ ]:
movies_with_new_rating = rate_10_products('movielens')
movies_with_new_rating

In [ ]:
rating_matrix_item_ML = movies_with_new_rating.pivot_table(index = 'movie_id',columns = 'user_id', values = 'rating').fillna(0)
display(rating_matrix_item_ML)

### k = 3


In [ ]:
item_recom(rating_matrix_item_ML, 611, 5,'movielens',number_neighbors=3)

### k = 10


In [ ]:
item_recom(rating_matrix_item_ML, 611, 5,'movielens',number_neighbors=10)

### k = 30


In [ ]:
item_recom(rating_matrix_item_ML, 611, 5,'movielens',number_neighbors=30)

### k = 50


In [ ]:
item_recom(rating_matrix_item_ML, 611, 5,'movielens',number_neighbors=50)

### Jester


In [ ]:
jokes_with_new_rating = rate_10_products('jester')
jokes_with_new_rating

In [ ]:
rating_matrix_item_JT_new = jokes_with_new_rating.pivot_table(index = 'joke_id',columns = 'user_id', values = 'rating').fillna(0)
display(rating_matrix_item_JT_new)

### k = 3

In [ ]:
item_recom(rating_matrix_item_JT_new, 2000, 5,'jester',number_neighbors=3)

### k = 10

In [ ]:
item_recom(rating_matrix_item_JT_new, 2000, 5,'jester',number_neighbors=10)

### k = 30

In [ ]:
item_recom(rating_matrix_item_JT_new, 2000, 5,'jester',number_neighbors=30)

### k = 50

In [ ]:
item_recom(rating_matrix_item_JT_new, 2000, 5,'jester',number_neighbors=50)

La segunda parte de la tarea constará de las siguientes secciones: 
1. Implementación de un sistema de recomendación de filtro colaborativo basado en ítems utilizando los k ítems más cercanos. Para esta parte, se debe utilizar <b> similaridad coseno </b> como medida de similaridad entre los ítems. Además, el parámetro k debe ser escogido por ustedes. 

2. Se ingresa nuevo usuario al sistema, se le pide que califique 10 productos (a elección) y a partir de eso se le realiza la recomendación de 5 productos que no ha calificado. Para poder llevar a cabo la recomendación recordar los siguientes pasos:
    >a. Se debe generar la matriz de similaridad entre productos basada en la similaridad coseno.
    
    >b. Para realizar la recomendación, se debe predecir el rating de todos los productos que el usuario nuevo del sistema no haya calificado aún. Para esto, se deben obtener los k ítems más cercanos al ítem a predecir y se debe predecir su rating en base a la <i> weighted sum </i> de los ratings de dichos k ítems. 
    
    >c. Una vez predecidos los ratings para todos los productos sin calificación, se deben recomendar al usuario los 5 productos con mayor rating predecido. 

## 3. Concluir y responder las siguientes preguntas: 

   >a. ¿Cuáles fueron las 5 recomendaciones obtenidas? Analice y concluya respecto a estos resultados.  
    
   >b. ¿Qué cantidad de vecinos cercanos (k) se escogió para la recomendación? ¿En qué influye la elección de este parámetro? ¿Qué sucede a medida que aumento este parámetro?
    
   >c. ¿Cuáles son las ventajas de este enfoque respecto al anterior? 


### a.- 
Con un k inicial de 3 vecinos cercanos estos fueron los resultados para ambos dataset:

In [ ]:
item_recom(rating_matrix_item_ML, 611, 5,'movielens',number_neighbors=3)

In [ ]:
item_recom(rating_matrix_item_JT_new, 2000, 5,'jester',number_neighbors=3)

Lo que podemos ver es que en ambos casos los valores de predicción son o muy buenos o muy malos, esto se debe al pequeño tamaño de muestra de donde se sacan estas recomendaciones. Por lo que se espera que a una muestra de mayor tamaño esto cambie y existan valores medios.

### b.- 
Luego de una variacíon de los k vecinos, podemos ver que para ambos dataset al momento de aumentar el k existen recomendaciones que siguen apareciendo, aunque son pocas. Lo que si ocurre es que las recomendaciones a medida que se ven mas vecinos los valores de predicción de las sugerencias bajan pero se "estabilizan" al comparar con más opciones, por lo que la sugerecia deja de ser o muy buena o muy mala y entra a valores medios, como en el caso de lso chistes con k = 50 que los valores de predicción tiende a 3.

### c.- 
La precisión de predicción es mejor en item based que en user based, además que mas fácil de explicar al usuario de donde viene la sugerencia. La consistencia de la sugerencia es mejor ya que se calcula respecto al puntaje de el mismo usuario. Otro factor de ventaja es que factor Sparsity del otro metodo, ya que el ratio de personas que evaluan items es muy bajo.

## 3. Conclusiones finales (20 puntos)

1. Analice los resultados obtenidos y contraste ambos enfoques (ubcf y ibcf). 
2. Analice ventajas y desventajas de ambos enfoques. 

1.- 
Como se pudo ver a lo largo del escrito, podemos ver que los resultados entregados por ubcf son poco variables ya que la sparsity por ejemplo del dataset movies, es muy alta, del 98%, lo que nos dice que la evaluación por parte de los users es muy baja, lo que afecta a la calidad de recomendación. 

2.- 
La ventaja de ubcf la variedad de sugerencias crece a medida que aumenta la cantidad de vecinos que se comparan. El gran problema de este enfoque es el valor del sparsity y como consecuencia el comienzo de los dataset, ya que al iniciar los datos las calificaciones son practicamente nulas, lo que afecta el sistema de sugerencias. Relacionado con lo anterior existe el problema de la escalabilidad.

Ibcf:
Since the ratings are predicted using the ratings of user herself, the predicted ratings tend to be much more consistent with the other ratings
of this user.
Item based similarity is a common choice for high load services.
>> The reason is that usually item’s neighbourhood changes much slower in compared to user’s neighbourhood.
Much easier to explain the recommendation to the users.
Item-based recommenders perform considerably better than the user-based ones.
The greater prediction accuracy of the item-based method is its main advantage.
Item-based methods are also more robust to shilling attacks
in recommender systems.

Item-based methods might sometimes recommend obvious items, or items which are not novel from previous user experiences.
LSA has an overall negative effect on the item-based recommendations.
